# Memory Optimized FinBERT Profiling: FP16 and AMP



Note: **AMP and FP16 are only enabled on CUDA by default** (to avoid MPS/CPU autocast edge cases).



## Imports
We import the necessary libraries and modules, including the custom `finbert` modules we have defined that allow for profiling.

In [ ]:
from __future__ import annotations

from pathlib import Path
import shutil
import time
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report
from torch.nn import CrossEntropyLoss
from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data
import finbert.utils as tools

import wandb

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

In [ ]:
# Paths
cl_path = project_dir / 'models' / 'mem_opt_comparison'
cl_data_path = project_dir / 'data' / 'sentiment_data'

# Clean up previous run
try:
    shutil.rmtree(cl_path)
except:
    pass

# W&B
wandb.init(
    entity="si2449-columbia-university",
    project="finbert-experiments",
    name="mem-opt-comparison",
    group="mem_optimization"
)

In [ ]:
USE_AMP = False

In [ ]:
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased', cache_dir=None, num_labels=3
)

config_baseline = Config(
    data_dir=cl_data_path,
    bert_model=bertmodel,
    num_train_epochs=6,
    model_dir=cl_path,
    max_seq_length=64,
    train_batch_size=32,
    learning_rate=0.00001420326287435756,
    output_mode='classification',
    warm_up_proportion=0.14386028719686458,
    local_rank=-1,
    discriminate=False,
    gradual_unfreeze=False,
    use_amp=False,  # Baseline uses FP32
)
config_baseline.profile_train_steps = 20

finbert = ProfiledFinBert(config_baseline)
finbert.base_model = 'bert-base-uncased'
finbert.prepare_model(label_list=['positive', 'negative', 'neutral'])

train_data = finbert.get_data('train')
test_data = finbert.get_data('test')

model = finbert.create_the_model()

# Train
start = time.perf_counter()
trained_model = finbert.train(train_examples=train_data, model=model)
baseline_train_wall_s = time.perf_counter() - start

In [ ]:
def timed_eval(*, finbert, model, examples, use_amp=False):
    """Evaluation with timing."""
    loader = finbert.get_loader(examples, phase="eval")
    device = finbert.device
    model.eval()
    
    preds, labels = [], []
    if device.type == "cuda":
        torch.cuda.synchronize(device)
    
    start = time.perf_counter()
    with torch.no_grad():
        for batch in loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, _ = batch
            
            if use_amp and device.type == "cuda":
                with torch.amp.autocast('cuda'):
                    logits = model(input_ids, attention_mask, token_type_ids)[0]
            else:
                logits = model(input_ids, attention_mask, token_type_ids)[0]
            
            preds.extend(logits.detach().cpu().numpy())
            labels.extend(label_ids.detach().cpu().numpy().tolist())
    
    if device.type == "cuda":
        torch.cuda.synchronize(device)
    
    wall_s = time.perf_counter() - start
    n = len(labels)
    
    return pd.DataFrame({"predictions": preds, "labels": labels}), {
        "eval_wall_s": wall_s,
        "eval_samples_per_s": n / wall_s if wall_s > 0 else float("inf"),
    }

def get_metrics(df):
    """Extract accuracy and F1 from eval results."""
    preds = np.array([np.argmax(p) for p in df['predictions']])
    labels = np.array(df['labels'])
    acc = (preds == labels).mean()
    from sklearn.metrics import f1_score
    f1 = f1_score(labels, preds, average='macro')
    return {"accuracy": acc, "f1_macro": f1}

# Evaluate baseline
baseline_eval_df, baseline_timing = timed_eval(finbert=finbert, model=trained_model, examples=test_data, use_amp=False)
baseline_metrics = get_metrics(baseline_eval_df)

print(f"Baseline - Accuracy: {baseline_metrics['accuracy']:.4f}, F1: {baseline_metrics['f1_macro']:.4f}")
print(f"Baseline - Throughput: {baseline_timing['eval_samples_per_s']:.1f} samples/sec")

In [ ]:
device = finbert.device
all_results = []

# ===== BASELINE =====
all_results.append({
    "variant": "baseline",
    "model_size_mb": get_model_size_mb(trained_model),
    **baseline_timing,
    **baseline_metrics,
})

# ===== FP16 WEIGHTS =====
if device.type == "cuda":
    # Load model with FP16 weights
    fp16_model = AutoModelForSequenceClassification.from_pretrained(
        cl_path, num_labels=3, torch_dtype=torch.float16
    ).to(device)
    
    fp16_eval_df, fp16_timing = timed_eval(finbert=finbert, model=fp16_model, examples=test_data, use_amp=False)
    fp16_metrics = get_metrics(fp16_eval_df)
    
    all_results.append({
        "variant": "fp16_weights",
        "model_size_mb": get_model_size_mb(fp16_model),
        **fp16_timing,
        **fp16_metrics,
    })
    print(f"FP16 - Accuracy: {fp16_metrics['accuracy']:.4f}, F1: {fp16_metrics['f1_macro']:.4f}")
    print(f"FP16 - Throughput: {fp16_timing['eval_samples_per_s']:.1f} samples/sec")
else:
    print("Skipping FP16 (requires CUDA)")

# ===== AMP AUTOCAST =====
if device.type == "cuda":
    # Use baseline model with AMP autocast during inference
    amp_eval_df, amp_timing = timed_eval(finbert=finbert, model=trained_model, examples=test_data, use_amp=True)
    amp_metrics = get_metrics(amp_eval_df)
    
    all_results.append({
        "variant": "amp_autocast",
        "model_size_mb": get_model_size_mb(trained_model),
        **amp_timing,
        **amp_metrics,
    })
    print(f"AMP - Accuracy: {amp_metrics['accuracy']:.4f}, F1: {amp_metrics['f1_macro']:.4f}")
    print(f"AMP - Throughput: {amp_timing['eval_samples_per_s']:.1f} samples/sec")
else:
    print("Skipping AMP (requires CUDA)")

results_df = pd.DataFrame(all_results)
results_df

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Throughput
ax = axes[0]
results_df.set_index("variant")["eval_samples_per_s"].plot(kind="bar", ax=ax, color=['#2ecc71', '#3498db', '#e74c3c'])
ax.set_title("Inference Throughput")
ax.set_ylabel("samples/sec")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

# Accuracy
ax = axes[1]
results_df.set_index("variant")["accuracy"].plot(kind="bar", ax=ax, color=['#2ecc71', '#3498db', '#e74c3c'])
ax.set_title("Accuracy")
ax.set_ylabel("accuracy")
ax.set_ylim(0.7, 1.0)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

# Model Size
ax = axes[2]
results_df.set_index("variant")["model_size_mb"].plot(kind="bar", ax=ax, color=['#2ecc71', '#3498db', '#e74c3c'])
ax.set_title("Model Size")
ax.set_ylabel("MB")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

# Speedup summary
if len(results_df) > 1:
    base = results_df[results_df["variant"] == "baseline"].iloc[0]
    print("\n=== Speedup vs Baseline ===")
    for _, row in results_df.iterrows():
        if row["variant"] != "baseline":
            speedup = row["eval_samples_per_s"] / base["eval_samples_per_s"]
            delta_acc = row["accuracy"] - base["accuracy"]
            print(f"{row['variant']}: {speedup:.2f}x speedup, Δaccuracy={delta_acc:+.4f}")

In [ ]:
# Log summary to W&B
summary = {
    "baseline_train_wall_s": baseline_train_wall_s,
    "baseline_accuracy": baseline_metrics["accuracy"],
    "baseline_f1": baseline_metrics["f1_macro"],
    "baseline_throughput": baseline_timing["eval_samples_per_s"],
}

if device.type == "cuda":
    summary.update({
        "fp16_accuracy": fp16_metrics["accuracy"],
        "fp16_throughput": fp16_timing["eval_samples_per_s"],
        "amp_accuracy": amp_metrics["accuracy"],
        "amp_throughput": amp_timing["eval_samples_per_s"],
    })

wandb.log(summary)
wandb.finish()

print("✓ Done!")